In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/My Drive/589MiniProject/")
%cd /content/drive/My Drive/589MiniProject/

Mounted at /content/drive
/content/drive/My Drive/589MiniProject


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from sklearn.linear_model import ElasticNet

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
from tqdm import tqdm

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 75

print('using device:', device)

using device: cpu


In [5]:
!pip install img2vec_pytorch

  Created wheel for img2vec-pytorch: filename=img2vec_pytorch-0.2.5-py3-none-any.whl size=4345 sha256=e8aaafe30c51b5cf842ecae5af433a0af1f35a4f142d3b353ac0378250e4e01d
  Stored in directory: /root/.cache/pip/wheels/07/4d/bf/9de03c3e76c20899b69498f90624ec499c7c67dd13f183f4e9
Successfully built img2vec-pytorch


In [6]:
from img2vec_pytorch import Img2Vec
from PIL import Image

def imgVec(folder='Train'):
  dataPath = "Data/" + folder + "/"
  metadataPath = "Data/MetaData/" + folder.lower() + ".csv"
  df = pd.read_csv(metadataPath)
  df['img_path'] = './Data/' + folder + "/" + df['Id'] + '.jpg'
  img2vec = Img2Vec(cuda=(torch.cuda.is_available()))  
  img_vecs = []
  for idx, row in df.iterrows():
    img = Image.open(row["img_path"])
    vec = img2vec.get_vec(img)
    img_vecs.append(vec)

  df['feature_vector'] = img_vecs
  return df


In [7]:
train_df = imgVec('Train')
train_df.reset_index(inplace=True, drop = True)
col_names = ["feat"+str(i) for i in range(len(train_df.iloc[0]["feature_vector"]))]
train_df[col_names] = pd.DataFrame(train_df.feature_vector.tolist(), index= train_df.index)
train_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True, errors = "ignore")


valid_df = imgVec('Valid')
valid_df.reset_index(inplace=True, drop = True)
col_names = ["feat"+str(i) for i in range(len(valid_df.iloc[0]["feature_vector"]))]
valid_df[col_names] = pd.DataFrame(valid_df.feature_vector.tolist(), index= valid_df.index)
valid_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True, errors = "ignore")

test_df = imgVec('Test')
test_df.reset_index(inplace=True, drop = True)
col_names = ["feat"+str(i) for i in range(len(test_df.iloc[0]["feature_vector"]))]
test_df[col_names] = pd.DataFrame(test_df.feature_vector.tolist(), index= test_df.index)
test_df.drop(columns = ["Id","img_path","feature_vector"], inplace=True, errors = "ignore")

print("Train ...")
print(train_df.head())

print("Valid ...")
print(valid_df.head())

print("Test ...")
print(test_df.head())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Train ...
   Subject Focus  Eyes  Face  Near  ...   feat508   feat509   feat510   feat511
0              0     1     1     1  ...  1.552251  0.729470  1.408372  1.468240
1              0     1     1     1  ...  0.841265  1.340379  2.015083  0.379474
2              0     1     1     1  ...  0.902725  0.322946  1.937619  0.461102
3              0     1     1     1  ...  1.789118  1.036564  0.068066  0.716284
4              0     1     1     1  ...  3.102909  1.097454  0.581716  0.273106

[5 rows x 525 columns]
Valid ...
   Subject Focus  Eyes  Face  Near  ...   feat508   feat509   feat510   feat511
0              0     0     1     1  ...  1.847430  0.734269  0.584828  1.115096
1              0     1     1     1  ...  1.499832  2.500482  0.277388  0.397272
2              0     1     1     0  ...  0.401953  0.236270  0.914187  0.319195
3              0     1     1     1  ...  2.246782  1.693573  0.013458  1.673211
4              0     1     1     1  ...  2.828182  1.301023  0.482935  0.697

In [8]:
def hyper_parameter_tuning(model_class, param_grid, train_X, train_y, valid_X, valid_y):
    """
    Tune the hyper-parameter using training and validation data

    Params
    ------
    model_class:    the model class
    param_grid:     the hyper-parameter grid, dict
    train:          the training data (train_X, train_y)
    valid:          the validatation data (valid_X, valid_y)

    Return
    ------
    model:          model fit with best params
    best_param:     the best params
    """

    # Set up the parameter grid
    param_grid = list(ParameterGrid(param_grid))
    best_model = None
    best_mse = float('inf')
    best_params = None

    # train the model with each parameter setting in the grid
    for params in param_grid:
        model = model_class(alpha=params['alpha'],l1_ratio = params['l1_ratio'],max_iter=params['max_iter'])
        model.fit(train_X,train_y)
        y_pred = model.predict(valid_X)
        mse = mean_squared_error(valid_y,y_pred)
        print(params,mse)
        if mse < best_mse:
            best_model = model
            best_mse = mse
            best_params = params

    # choose the model with lowest MAE on validation set
    # then fit the model with the training and validation set (refit)

    full_training_set_X = np.concatenate([train_X, valid_X], axis=0)
    full_training_set_y = np.concatenate([train_y, valid_y], axis=0)
    best_model.fit(full_training_set_X,full_training_set_y)

    # return the fitted model and the best parameter setting
    return best_model, best_params

In [9]:
train_y, valid_y, test_y = train_df['Pawpularity'], valid_df['Pawpularity'], test_df['Pawpularity']
train_X = train_df.drop(columns=['Pawpularity'])
train_X = preprocessing.StandardScaler().fit_transform(train_X)
valid_X = valid_df.drop(columns=['Pawpularity'])
valid_X = preprocessing.StandardScaler().fit_transform(valid_X)
test_X = test_df.drop(columns=['Pawpularity'])
test_X = preprocessing.StandardScaler().fit_transform(test_X)

In [10]:
elasticnet_model = ElasticNet()
elasticnet_model.fit(train_X, train_y)
validpred_y = elasticnet_model.predict(valid_X)
testpred_y = elasticnet_model.predict(test_X)

validmse = mean_squared_error(valid_y, validpred_y)
testmse = mean_squared_error(test_y, testpred_y)

print("Validation MSE ... ", validmse)
print("TEST MSE ... ", testmse)

Validation MSE ...  372.84315736939084
TEST MSE ...  352.9757184171703


In [11]:
elasticnet_grid = dict(alpha=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  l1_ratio = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  max_iter =[1000, 2000, 5000, 10000, 20000])
elasticnet_model, elasticnet_param = hyper_parameter_tuning(ElasticNet, elasticnet_grid, train_X, train_y, valid_X, valid_y)
print('best parameter setting for Ridge Regression:',elasticnet_param)
testpred_y = elasticnet_model.predict(test_X)
testmse = mean_squared_error(test_y, testpred_y)
print("TEST MSE ... ", testmse)
print("TEST RMSE ...", np.sqrt(testmse))

{'alpha': 0.1, 'l1_ratio': 0, 'max_iter': 1000} 367.7929424001861
{'alpha': 0.1, 'l1_ratio': 0, 'max_iter': 2000} 367.79191957734537
{'alpha': 0.1, 'l1_ratio': 0, 'max_iter': 5000} 367.7919679079294
{'alpha': 0.1, 'l1_ratio': 0, 'max_iter': 10000} 367.79196790975635
{'alpha': 0.1, 'l1_ratio': 0, 'max_iter': 20000} 367.7919679097563
{'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 1000} 367.21660405413394
{'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 2000} 367.21803151756853
{'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 5000} 367.21803151756853
{'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 10000} 367.21803151756853
{'alpha': 0.1, 'l1_ratio': 0.1, 'max_iter': 20000} 367.21803151756853
{'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 1000} 366.7554531231936
{'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 2000} 366.7554531231936
{'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 5000} 366.7554531231936
{'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 10000} 366.7554531231936
{'alpha': 0.1, 'l1_ratio': 0.2, 'max_iter': 20